In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
#import pickle
import random
import itertools
dunnhumby_path = "/Users/johnabraham/Desktop/dunnhumby_The-Complete-Journey/dunnhumby - The Complete Journey CSV"

In [3]:
trans_data = pd.read_csv(dunnhumby_path+"/transaction_data.csv")

In [4]:
# Make everything in lower case
trans_data.rename(columns=dict(zip(trans_data.columns,trans_data.columns.str.lower())),inplace=True)

In [87]:
trans_data_grpby = trans_data.groupby(['household_key','product_id'],as_index=False)['sales_value','quantity'].agg({'sales_value':'sum','quantity':'sum'})

# We need to ensure products that are rarely bought by anyone should not be found similar with high confidence
# Keeping a threshold that at least 3 people have to purchase an item for it to go into the collab filtering
allowed_product_list = trans_data_grpby.groupby('product_id',as_index=False)['household_key'].agg({'household_key':'count'})
allowed_product_list = list(allowed_product_list[allowed_product_list.household_key>20]['product_id'])
trans_data_grpby = trans_data_grpby[trans_data_grpby.product_id.isin(allowed_product_list)]
# Note that this filter must have the same value as is there in the Recommendations.R file. Otherwise it will throw a KeyError in the part where
# recommendations are made
# This is to avoid costly things that are not frequently purchased
trans_data_grpby =trans_data_grpby[trans_data_grpby.quantity>3]
#trans_data_cross = trans_data_grpby.pivot_table(index='household_key',columns='product_id', values='sales_value').reset_index()

In [88]:
hh_key_idx = dict(zip(trans_data_grpby.household_key.unique(),np.arange(len(trans_data_grpby.household_key.unique()))))
prod_id_idx = dict(zip(trans_data_grpby.product_id.unique(), np.arange(len(trans_data_grpby.product_id.unique()))))
prod_idx_2_id = dict(enumerate(trans_data_grpby.product_id.unique()))

In [89]:
hh_prod_mat = csr_matrix((trans_data_grpby.sales_value,
                          (list(map(lambda x: hh_key_idx[x],trans_data_grpby.household_key)),
                           list(map(lambda x: prod_id_idx[x],trans_data_grpby.product_id)))),
                          shape=(len(hh_key_idx),len(prod_id_idx)))

In [90]:
hh_prod_sim = cosine_similarity(hh_prod_mat.transpose())
hh_prod_sim.shape

(12954, 12954)

In [8]:
# Earlier planned to write to a file to read in R. But the file is just too big (beyond 1 GB)
# hh_prod_sim_df = pd.DataFrame(hh_prod_sim)
# hh_prod_sim_df.columns=trans_data_grpby.product_id.unique()
# hh_prod_sim_df['product_id']=trans_data_grpby.product_id.unique()

In [9]:
# Output the file as a pickle file - Kernal dying
# with open("/Users/johnabraham/Desktop/dunnhumby_The-Complete-Journey/ProductRecommendation.P","wb") as file:
#     pickle.dump(hh_prod_sim_df,file)

### Try to manually see how highly similar products look

In [96]:
# Read in the product fille 
product = pd.read_csv(dunnhumby_path+"/product.csv")
num_prods_2_inspect = 4
num_sim_prods = 4
product_list = np.random.choice(list(prod_id_idx.keys()),size=num_prods_2_inspect,replace=False)
#list(product.sample(num_prods_2_inspect).PRODUCT_ID) # Not all products are in the collab filtering matrix
display(product_list)
for product_id in product_list:
    prod_n_sim_list = np.argsort(-hh_prod_sim[prod_id_idx[product_id],:])[:num_sim_prods+1]
    similarity = hh_prod_sim[prod_id_idx[product_id],prod_n_sim_list]
    prod_ids = [prod_idx_2_id[prod_idx] for prod_idx in prod_n_sim_list]
    prod_names = [(product[product.PRODUCT_ID==prod_id]['SUB_COMMODITY_DESC']).to_string(index=False) for prod_id in prod_ids]
    display(pd.DataFrame({'product_id':prod_ids,'similarity':similarity,'prod_names':prod_names}))


array([ 1108046,   948966, 12330860,  9296795])

,prod_names,product_id,similarity
0,ORAL HYGIENE BRUSHES,1108046,1.000000
1,ORAL HYGIENE BRUSHES,1099397,1.000000
2,CEREAL BARS,906511,0.932511
3,SNACK CRACKERS,979523,0.867935
4,PIZZA/PREMIUM,834572,0.847278


,prod_names,product_id,similarity
0,SOFT DRINKS 20PK&24PK CAN CARB,948966,1.000000
1,SOFT DRINKS 6PK/4PK CAN CARB (,6704144,0.847998
2,SFT DRNK MLT-PK BTL CARB (EXCP,9803819,0.834019
3,SFT DRNK MLT-PK BTL CARB (EXCP,1132956,0.700076
4,TABLE SALT POPCORN SALTICE C,999318,0.657381


,prod_names,product_id,similarity
0,TORTILLA/NACHO CHIPS,12330860,1.000000
1,SHAMPOO,13095304,0.816497
2,APPLES GALA (BULK&BAG),16729363,0.816497
3,CHILDREN S ACTIVITY,911093,0.816497
4,SHAMPOO,1126492,0.816497


,prod_names,product_id,similarity
0,SCOOP LITTER,9296795,1.000000
1,CANDY BAGS-NON CHOCOLATE,1053420,0.751491
2,CHEESE: NATURAL BULK,1020322,0.751491
3,TOOTHPASTE,968522,0.751491
4,DRY SOUP,874998,0.751491


### This part below expects the R-code to generate recommendation_output.csv is run and that file is available at this location

In [65]:
processed_files_path = "/Users/johnabraham/Dropbox/MSARelated/Spring2018/MGT6203/ProjectRelated"

In [74]:
# Read a dataset that has store_id, household_id, product_id from the historical data and use the collaborative filtering based method to append more products
# Return the dataset along with a key that says which one it is based on - historical or recommendation
reco_data = pd.read_csv(processed_files_path+"/recommendation_output.csv")
reco_data.head()

,store_id,household_key,recommendation_rank,product_id
0,292,1397,1,918046
1,292,1397,3,1050482
2,292,1397,4,1085983
3,292,1397,5,1106523
4,292,1397,2,8293425


In [79]:
reco_data_augmented = reco_data[['store_id','household_key','product_id']]
num_reco_products = 3 # Only up to num_reco_products will be added
similarity_threshold = 0.4 # Only products with a similarity of similarity_threshold or above will be considered 'similar'
for _,record in reco_data.iterrows():
    if record['product_id'] in prod_id_idx:
        prod_idx=prod_id_idx[record['product_id']]
        similarity_scores = hh_prod_sim[prod_idx,:]
        recommended_additional_product_idx = np.argsort(similarity_scores)[-(num_reco_products+1):-1]
        for prod_idx in recommended_additional_product_idx:
            if(similarity_scores[prod_idx]>=similarity_threshold):
                reco_data_augmented=reco_data_augmented.append(pd.DataFrame({"store_id":record['store_id'],"household_key":record['household_key'],
                                                                           'product_id':prod_idx_2_id[prod_idx]},index=[0]),ignore_index=True)
reco_data_augmented=reco_data_augmented.drop_duplicates()
reco_data_augmented.to_csv(processed_files_path+"/recommendation_augmented_output.csv",index=False)